<a href="https://colab.research.google.com/github/jembi/mpi-toolkit-notebook/blob/cr-review-dataset/fastLink-notebook/Linking/FastLinkRecordLinking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing rpy2 
rpy2 is running an embedded R, providing access to it from Python using R’s own C-API through either:

a high-level interface making R functions an objects just like Python functions and providing a seamless conversion to numpy and pandas data structures

a low-level interface closer to the C-API

In [1]:
!pip install rpy2

## Import python packages

In [2]:
import pandas as pd
import logging
from rpy2.robjects import globalenv
from rpy2.robjects.vectors import StrVector
import rpy2.robjects as r_objects
import rpy2.robjects.packages as r_packages
r = r_objects.r

from ipywidgets import Dropdown
from ipywidgets import FloatSlider

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO)
logging.info('Started')

2021-07-29 13:02:37,566 INFO:Started


## Import R packages

In [3]:
utils = r_packages.importr('utils')
utils.chooseCRANmirror(ind=1)
pack_names = ('fastLink', 'tictoc', 'strex', 'data.table', 'csv', 'stringr')
names_to_install = [x for x in pack_names if not r_packages.isinstalled(x)]
if len(names_to_install) > 0:
  utils.install_packages(StrVector(names_to_install))
  base = r_packages.importr('base')
  stats = r_packages.importr('stats')
  fastLink = r_packages.importr('fastLink')
  strex = r_packages.importr('strex')
  data_table = r_packages.importr('data.table')
  stringr = r_packages.importr('stringr')

2021-07-29 13:02:47,054 WARNING:R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

2021-07-29 13:02:48,133 WARNING:R[write to console]: also installing the dependencies ‘pixmap’, ‘sp’, ‘iterators’, ‘ade4’, ‘ggrepel’, ‘xtable’, ‘scatterplot3d’, ‘lpSolve’, ‘foreach’, ‘doParallel’, ‘gtools’, ‘stringdist’, ‘FactoClass’, ‘adagio’, ‘plotrix’, ‘RcppArmadillo’, ‘RcppEigen’, ‘checkmate’


2021-07-29 13:02:48,139 WARNING:R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/pixmap_0.4-12.tar.gz'

2021-07-29 13:02:48,943 WARNING:R[write to console]: Content type 'application/x-gzip'
2021-07-29 13:02:48,945 WARNING:R[write to console]:  length 34637 bytes (33 KB)

2021-07-29 13:02:48,947 WARNING:R[write to console]: =
2021-07-29 13:02:48,949 WARNING:R[write to console]: =
2021-07-29 13:02:48,952 WARNING:R[write to console]: =
2021-07-29 13:02:48,953 WARNING:R[write to console]: =
2021-07-29 13:02:48,955 WARNING:R[write to con

## Upload a file from your computer

In [4]:
from google.colab import files
uploaded = files.upload()


Saving data-50-25.csv to data-50-25.csv


## Reading a csv file and splitting it into 2 dataframes

In [5]:

globalenv['csv'] = r['read.csv']("data-50-25.csv", header=True, stringsAsFactors=False)
r('csv[csv==""] <- NA')
#r('csv$group    <- as.integer(substr(csv$ID, 5, 12))')
col_names_r = r('colnames(csv)')
col_names = list(col_names_r)
r('dfA          <- csv[str_detect(csv$ID, "-aaa-"), ]')
r('dfB          <- csv[str_detect(csv$ID, "-bbb-"), ]')
s = r('structure(list(csv = csv, dfA = dfA, dfB = dfB))')

menu_1 = Dropdown(description="Choose your unique identifier", options=col_names)
menu_2 = Dropdown(description="Choose your string distance algorithm", options=["Jaro-Winkler", "Levensthein"])
slider_1 = FloatSlider(description="cut.a:", value=0.94, min=0, max=1, step=0.01)
slider_2 = FloatSlider(description="cut.p:", value=0.88, min=0, max=1, step=0.01)
display(menu_1, menu_2, slider_1, slider_2)


Dropdown(description='Choose your unique identifier', options=('ID', 'hivCaseReportNumber', 'name', 'fathersNa…

Dropdown(description='Choose your string distance algorithm', options=('Jaro-Winkler', 'Levensthein'), value='…

FloatSlider(value=0.94, description='cut.a:', max=1.0, step=0.01)

FloatSlider(value=0.88, description='cut.p:', max=1.0, step=0.01)

## Get user input
The Jaro–Winkler distance is a string metric measuring an edit distance between two sequences.
The lower the Jaro–Winkler distance for two strings is, the more similar the strings are. The score is normalized such that 0 means an exact match and 1 means there is no similarity. 

The Levenshtein distance is a string metric for measuring the difference between two sequences. Informally, the Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.


In [6]:
key = menu_1.value
string_distance = menu_2.value
if string_distance == "Jaro-Winkler":
  string_distance = "jw"
else:
  string_distance = "lv"
cut_a = slider_1.value
cut_p = slider_2.value
print(key)
print(string_distance)
print(cut_a)
print(cut_p)

ID
jw
0.94
0.88


## fastLink: Fast Probabilistic Record Linkage with Missing Data
Implements a Fellegi-Sunter probabilistic record linkage model that allows for missing data and the inclusion of auxiliary information. This includes functionalities to conduct a merge of two datasets under the Fellegi-Sunter model using the Expectation-Maximization algorithm. In addition, tools for preparing, adjusting, and summarizing data merges are included.

## Link the 2 dataframes using fastLink

In [86]:
  logging.info("FastLink : initialized")

  df_a = r('dfA')
  df_b = r('dfB')

  def fl_link(df_a, df_b, key, string_distance, cut_a, cut_p):
      get_links = r('''
          my_fl_link <- function(dfA, dfB) {{
              pasteT <- function(x) {{
                  x <- sort(x)
                  x <- paste(x, collapse = ",")
                  x
              }}
              
              varnames <- colnames(dfA[,-1])
              #varnames <- varnames[-which(varnames %in% c('{0}'))]
              print(varnames)
              fl_out <- fastLink(dfA = dfA, dfB = dfB, varnames = varnames,
                                  stringdist.match = varnames, stringdist.method = '{1}', cut.a = {2}, cut.p = {3},
                                  dedupe.matches = FALSE, linprog.dedupe = FALSE,
                                  cond.indep = TRUE,
                                  n.cores = 8,
                                  verbose = TRUE)
              inds_ab <- data.table(cbind(fl_out$matches$inds.a, fl_out$matches$inds.b))
              inds_ab[, `:=`(V3, pasteT(V2)), by = V1]
              inds_ab <- inds_ab[,.(V1, V3)]
              inds_ab <- inds_ab[!duplicated(inds_ab)]
              setnames(inds_ab, 'V3', 'V2')
              structure(list(fl_out = fl_out, inds_ab = inds_ab))
          }}'''.format(key, string_distance, cut_a, cut_p))
      return get_links(df_a, df_b)

2021-07-29 16:31:57,932 INFO:FastLink : initialized


## Display fastLink Logs to the console

In [8]:
def analytics(process):
  varnames = tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('EM').rx2('varnames'))
  logging.info(('{0}: %s'.format(process), tuple(globalenv['{0}'.format(process)].names)))
  logging.info('fl_out: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').names))
  logging.info('fl_out: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('matches').names))
  logging.info('EM: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('EM').names))
  logging.info('patterns.w: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('EM').rx2('patterns.w').names[1]))
  logging.info('varnames: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('EM').rx2('varnames')))
  logging.info('patterns: %s', tuple(globalenv['{0}'.format(process)].rx2('fl_out').rx2('patterns').names))
  logging.info('inds_ab: %s', tuple(globalenv['{0}'.format(process)].rx2('inds_ab').names))
  # The posterior probability of a pair matching.
  logging.info('p.m: %f', r('{0}$fl_out$EM$p.m'.format(process))[0])
  # The posterior probability of a pair not matching.
  logging.info('p.u: %f', r('{0}$fl_out$EM$p.u'.format(process))[0])
  # The posterior of the matching prVobability for a specific matching field.
  logging.info("EM.p.gamma.k.m")
  em_p_gamma_k_m = []
  for i in range(1, len(r('{0}$fl_out$EM$p.gamma.k.m'.format(process))) - 1):
    placeholder = ['p.gamma.k.m ----- %-20s : %3.10f  %3.10f  %3.10f',
                    varnames[i],
                    r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][0],
                    abs(r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][1]),
                    r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][0] + r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][1]]
    em_p_gamma_k_m.append(placeholder)
    logging.info('p.gamma.k.m ----- %-20s : %3.10f  %3.10f  %3.10f',
                  varnames[i],
                  r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][0],
                  abs(r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][1]),
                  r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][0] + r('{0}$fl_out$EM$p.gamma.k.m'.format(process))[i][1])
  # The posterior of the non-matching probability for a specific matching field.
  logging.info("EM.p.gamma.k.u")
  em_p_gamma_k_u = []
  for i in range(1, len(r('{0}$fl_out$EM$p.gamma.k.u'.format(process))) - 1):
    placeholder = ['p.gamma.k.u ----- %-20s : %3.10f  %3.10f  %3.10f',
                    varnames[i],
                    abs(r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][0]),
                    r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][1],
                    r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][0] + r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][1]]
    em_p_gamma_k_u.append(placeholder)
    logging.info('p.gamma.k.u ----- %-20s : %3.10f  %3.10f  %3.10f',
                  varnames[i],
                  abs(r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][0]),
                  r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][1],
                  r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][0] + r('{0}$fl_out$EM$p.gamma.k.u'.format(process))[i][1])
  logging.info('iter.converge: %d', r('{0}$fl_out$EM$iter.converge'.format(process))[0])
  return varnames, em_p_gamma_k_m, em_p_gamma_k_u

In [90]:
globalenv['links'] = fl_link(df_a, df_b, key, string_distance, cut_a, cut_p)
log_info = analytics('links')
varnames = log_info[0]
em_p_gamma_k_m = log_info[1]
em_p_gamma_k_u = log_info[2]

v1 = tuple(map(int, r('links$inds_ab$V1')))
v2 = tuple(r('links$inds_ab$V2'))
false_positives = 0
true_positives = 0

fields = ('ID',) + varnames
left = pd.DataFrame(columns=('key',) + fields)
right = pd.DataFrame(columns=('key',) + fields)
print(left)

k = 0
for i in range(len(v1)):
  dupe_links = tuple(map(int, v2[i].split(',')))
  master = r('dfA[{},]'.format(v1[i]))
  left.loc[i] = (i,) + tuple(map(lambda x: master.rx2(x)[0], fields))
  for j in range(len(dupe_links)):
    dup = r('dfB[{},]'.format(dupe_links[j]))
    right.loc[k] = (i,) + tuple(map(lambda x: dup.rx2(x)[0], fields))
    k = k + 1
    


 [1] "hivCaseReportNumber" "name"                "fathersName"        
 [4] "fathersFatherName"   "mothersName"         "mothersFatherName"  
 [7] "gender"              "dob"                 "district"           
[10] "town"                "PhoneNumber"        

fastLink(): Fast Probabilistic Record Linkage

If you set return.all to FALSE, you will not be able to calculate a confusion table as a summary statistic.
Calculating matches for each variable.
    Matching variable hivCaseReportNumber using string-distance matching.
    Matching variable name using string-distance matching.
    Matching variable fathersName using string-distance matching.
    Matching variable fathersFatherName using string-distance matching.
    Matching variable mothersName using string-distance matching.
    Matching variable mothersFatherName using string-distance matching.
    Matching variable gender using string-distance matching.
    Matching variable dob using string-distance matching.
    Matching va

2021-07-29 16:47:41,394 INFO:('links: %s', ('fl_out', 'inds_ab'))
2021-07-29 16:47:41,396 INFO:fl_out: ('matches', 'EM', 'patterns', 'posterior', 'nobs.a', 'nobs.b')
2021-07-29 16:47:41,398 INFO:fl_out: ('inds.a', 'inds.b')
2021-07-29 16:47:41,400 INFO:EM: ('zeta.j', 'p.m', 'p.u', 'p.gamma.k.m', 'p.gamma.k.u', 'p.gamma.j.m', 'p.gamma.j.u', 'patterns.w', 'iter.converge', 'nobs.a', 'nobs.b', 'varnames')
2021-07-29 16:47:41,402 INFO:patterns.w: ('gamma.1', 'gamma.2', 'gamma.3', 'gamma.4', 'gamma.5', 'gamma.6', 'gamma.7', 'gamma.8', 'gamma.9', 'gamma.10', 'gamma.11', 'counts', 'weights', 'p.gamma.j.m', 'p.gamma.j.u')
2021-07-29 16:47:41,404 INFO:varnames: ('hivCaseReportNumber', 'name', 'fathersName', 'fathersFatherName', 'mothersName', 'mothersFatherName', 'gender', 'dob', 'district', 'town', 'PhoneNumber')
2021-07-29 16:47:41,407 INFO:patterns: ('gamma.1', 'gamma.2', 'gamma.3', 'gamma.4', 'gamma.5', 'gamma.6', 'gamma.7', 'gamma.8', 'gamma.9', 'gamma.10', 'gamma.11')
2021-07-29 16:47:41,4

Empty DataFrame
Columns: [key, ID, hivCaseReportNumber, name, fathersName, fathersFatherName, mothersName, mothersFatherName, gender, dob, district, town, PhoneNumber]
Index: []


## Display result to the screen

In [92]:
print(f"left side {left}")
print(f"right side {right}")

left side    key                  ID  ...            town   PhoneNumber
0    0  rec-00000000-aaa-0  ...            Duna  011-655-1890
1    1  rec-00000001-aaa-0  ...            Sayo  033-619-0743
2    2  rec-00000005-aaa-0  ...     East Belesa  022-310-0181
3    3  rec-00000009-aaa-0  ...     Hawasa town  011-112-6142
4    4  rec-00000010-aaa-0  ...            Euwa  091-734-3701
5    5  rec-00000017-aaa-0  ...           Malga  022-630-0280
6    6  rec-00000018-aaa-0  ...     Abe Dongoro  011-655-0179
7    7  rec-00000022-aaa-0  ...  Chabe Gambeltu  011-318-4884
8    8  rec-00000023-aaa-0  ...       Robe Town  091-439-4997
9    9  rec-00000029-aaa-0  ...            Ezha  025-279-1520
10  10  rec-00000033-aaa-0  ...           Ada'a  091-463-8224
11  11  rec-00000036-aaa-0  ...     Tach Gayint  058-776-4810
12  12  rec-00000038-aaa-0  ...        Gunagado  011-651-8614
13  13  rec-00000039-aaa-0  ...          Gulele  098-119-5453
14  14  rec-00000040-aaa-0  ...      Adama town  011-652-529